In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
# Print column names to find desired
# Desired columns: koi_prad, koi_teq, koi_slogg, koi_period
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

In [4]:
# Set features. This will also be used as your x values.
selected_features = df[["koi_prad", "koi_teq", "koi_slogg", "koi_period","koi_disposition"]]
selected_features.head()

,koi_prad,koi_teq,koi_slogg,koi_period,koi_disposition
0,2.83,443,4.467,54.418383,CONFIRMED
1,14.60,638,4.544,19.899140,FALSE POSITIVE
2,33.46,1395,4.564,1.736952,FALSE POSITIVE
3,2.75,1406,4.438,2.525592,CONFIRMED
4,2.77,1160,4.486,4.134435,CONFIRMED


In [5]:
# Assign numeric values to confirmed koi_disposition data
cleaned = pd.get_dummies(selected_features)
cleaned = cleaned.drop(columns=['koi_disposition_CANDIDATE','koi_disposition_FALSE POSITIVE'])
cleaned.head()

,koi_prad,koi_teq,koi_slogg,koi_period,koi_disposition_CONFIRMED
0,2.83,443,4.467,54.418383,1
1,14.60,638,4.544,19.899140,0
2,33.46,1395,4.564,1.736952,0
3,2.75,1406,4.438,2.525592,1
4,2.77,1160,4.486,4.134435,1


In [6]:
# Reshape y values
target = cleaned["koi_disposition_CONFIRMED"].values.reshape(-1,1)
target.shape

(6991, 1)

In [7]:
# Drop y column for x values and check shape
data = cleaned.drop("koi_disposition_CONFIRMED", axis=1)
data.shape

(6991, 4)

In [8]:
# Test, train, split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [9]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [11]:
classifier.fit(X_train, y_train)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.7436582109479306
Testing Data Score: 0.7208237986270023


In [40]:
# Predict outcome using X_test
predictions = classifier.predict(X_test)

In [41]:
# Reshape y_test to fit in DataFrame
y_test = y_test.flatten()
y_test.shape

(1748,)

In [43]:
# Show predictions vs. actual outcomes
# 0 - koi disposition NOT confirmed, 1 - koi disposition confirmed

pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
1743,0,0
1744,0,0
1745,1,1
1746,0,1
